In [1]:
import numpy as np
import cv2
import os
import sys


#source = "in_06.png"
#target = "tar_06.png"
source = sys.argv[1]
target = sys.argv[2]

source = cv2.imread(source)
target = cv2.imread(target)



source = cv2.cvtColor(source, cv2.COLOR_BGR2RGB)
target = cv2.cvtColor(target, cv2.COLOR_BGR2RGB)


source = np.asfarray(source, dtype='float32')
target = np.asfarray(target, dtype='float32')


def color_transfer(source, target):



    source /= 255.0
    target /= 255.0

    source = np.maximum(source, 1/255)
    target = np.maximum(target, 1/255)

    a = np.array([[0.3811, 0.5783, 0.0402],[0.1967, 0.7244, 0.0782],[0.0241, 0.1288, 0.8444]])
    b = np.array([[1/np.sqrt(3), 0, 0],[0, 1/np.sqrt(6) , 0],[0, 0, 1/np.sqrt(2)]])
    c = np.array([[1, 1, 1],[1, 1, -2],[1, -1, 0]])
    b2 = np.array([[np.sqrt(3)/3, 0, 0],[0, np.sqrt(6)/6, 0],[0, 0, np.sqrt(2)/2]]) 
    c2 = np.array([[1, 1, 1],[1, 1, -1],[1, -2, 0]])
    c3 = np.array([[4.4679, -3.5873, 0.1193],[-1.2186, 2.3809, -0.1624],[0.0497, -0.2439, 1.2045]])

    LMS_s = source @ a 
    LMS_t = target @ a 

    LMS_s = np.log10(LMS_s)
    LMS_t = np.log10(LMS_t)

    mul_bc = b @ c  # multiplying b and c 
    lab_s  = LMS_s @ mul_bc 
    lab_t  = LMS_t @ mul_bc  

    l = lab_s[:, :, 0]
    a = lab_s[:, :, 1]
    b = lab_s[:, :, 2]

    l1 = lab_t[:, :, 0]
    a1 = lab_t[:, :, 1]
    b1 = lab_t[:, :, 2]



    (sl_mean, sl_std) = (np.mean(l),np.std(l))
    (sa_mean, sa_std) = (np.mean(a),np.std(a))
    (sb_mean, sb_std) = (np.mean(b),np.std(b))

    (tl_mean, tl_std) = (np.mean(l1),np.std(l1))
    (ta_mean, ta_std) = (np.mean(a1),np.std(a1))
    (tb_mean, tb_std) = (np.mean(b1),np.std(b1))
    
    # Subtract the means from the source image
    
    l -= sl_mean
    a -= sa_mean
    b -= sb_mean
    
    # Scale by the standard deviations
    
    l = (tl_std / sl_std) * l
    a = (ta_std / sa_std) * a
    b = (tb_std / sb_std) * b

    l += tl_mean
    a += ta_mean
    b += tb_mean
    

    transfer  = np.dstack((l,a,b))

    mul_b2c2 = b2 @ c2
    
    LMS_res  = transfer @ mul_b2c2
    
    LMS_res = np.power(10, LMS_res)
    
    result  = LMS_res @ c3

    result = result[:,:,::-1]
    
    return result


transferred = color_transfer(source, target)

cv2.imshow('result',transferred)
k = cv2.waitKey(0)

error: OpenCV(4.6.0) C:\b\abs_74oeeuevib\croots\recipe\opencv-suite_1664548340488\work\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
